# 2D FEM Assembly Layout Tutorial

This tutorial explains how the 2D FEM solver assembles sparse matrices and residual vectors using precomputed index structures.

We use a minimal 2x2 inner grid to make the assembly process manually traceable.

**Prerequisites:** Familiarity with FEM basics is helpful. See `08_FEM1d_theory.ipynb` for the underlying theory and term naming conventions.

## Overview

The assembly layout provides precomputed index mappings for O(nnz) matrix/vector assembly:

```
FEMAssemblyLayout (top-level container)
├── MatrixCOOPattern  - Sparse matrix structure (row/col indices)
├── RHSPattern        - RHS vector structure
├── jacobian_terms    - Dict[TermKey, JacobianTermMap]
└── rhs_terms         - Dict[TermKey, RHSTermMap]
```

## Class Reference

### MatrixCOOPattern

Stores the sparse matrix structure in COO (Coordinate) format.

| Attribute | Type | Description |
|-----------|------|-------------|
| `nnz` | int | Number of non-zero entries |
| `local_rows` | IntArray | Local row indices for each COO entry |
| `local_cols` | IntArray | Local column indices for each COO entry |
| `global_rows` | IntArray | Global row indices (for PETSc/MPI) |
| `global_cols` | IntArray | Global column indices (for PETSc/MPI) |

### RHSPattern

Stores the RHS vector assembly structure.

| Attribute | Type | Description |
|-----------|------|-------------|
| `size` | int | Local RHS size (nb_vars × nb_inner_pts) |
| `global_rows` | IntArray | Global row indices for PETSc assembly |

### JacobianTermMap

Maps contributions from one physics term to Jacobian matrix entries.

| Attribute | Type | Description |
|-----------|------|-------------|
| `coo_idx` | IntArray | Which COO entries receive contributions |
| `weights` | NDArray | FEM integration weights |
| `quad_idx` | IntArray | Quadrature point index (0-5) |
| `sq_x`, `sq_y` | IntArray | Square indices for field value lookup |
| `signs` | NDArray or None | Sign multipliers (±1) for derivative terms |

**Note on `signs`:** For terms with spatial derivatives (∂/∂x or ∂/∂y), the sign indicates whether the contribution comes from the "left" or "right" side of the finite difference stencil. For non-derivative terms, `signs` is `None`.

### RHSTermMap

Maps contributions from one physics term to RHS vector entries.

| Attribute | Type | Description |
|-----------|------|-------------|
| `output_idx` | IntArray | Which R[i] entries receive contributions |
| `weights` | NDArray | FEM integration weights |
| `quad_idx` | IntArray | Quadrature point index (0-5) |
| `sq_x`, `sq_y` | IntArray | Square indices for field value lookup |

### FEMAssemblyLayout

Top-level container that holds everything.

| Attribute | Type | Description |
|-----------|------|-------------|
| `matrix_coo` | MatrixCOOPattern | Sparse matrix structure |
| `rhs` | RHSPattern | RHS vector structure |
| `jacobian_terms` | dict | Maps (term, res, var) → JacobianTermMap |
| `rhs_terms` | dict | Maps (term, res) → RHSTermMap |
| `dx`, `dy` | float | Grid spacing |
| `nb_vars` | int | Number of variables (3 or 4) |
| `nb_inner_pts` | int | Number of inner grid points |
| `nb_global_pts` | int | Total global grid points |

---
## Minimal Example Setup

We create a 2×2 inner grid (4 inner points) with Dirichlet BCs on all sides.
This gives us:
- 4 inner points (where we solve)
- 3 variables (ρ, jx, jy) → 12 DOFs total
- 3×3 = 9 squares (spanning ghost and inner points), each split into 2 triangles → 18 triangles

In [ ]:
import numpy as np
from GaPFlow.problem import Problem

# Minimal 2x2 configuration
CONFIG = """
options:
    output: /tmp/fem2d_assembly_tutorial
    write_freq: 1000
    silent: True

grid:
    Lx: 0.1
    Ly: 0.1
    Nx: 2
    Ny: 2
    xE: ['D', 'N', 'N']
    xW: ['D', 'N', 'N']
    yS: ['D', 'N', 'N']
    yN: ['D', 'N', 'N']
    xE_D: 1.0
    xW_D: 1.0
    yS_D: 1.0
    yN_D: 1.0

geometry:
    type: inclined
    hmax: 1e-5
    hmin: 1e-5
    U: 0.0
    V: 0.0

numerics:
    solver: fem
    dt: 1e-8
    tol: 1e-6
    max_it: 100

properties:
    EOS: PL
    rho0: 1.0
    shear: 1e-3
    bulk: 0.
    P0: 101325
    alpha: 0.

fem_solver:
    type: newton_alpha
    dynamic: False
    equations:
        term_list: ['R11x', 'R11y', 'R21x', 'R21y', 'R24x', 'R24y']
"""

problem = Problem.from_string(CONFIG)
solver = problem.solver

# Initialize solver steps individually to access assembly_layout BEFORE PETSc modifies it
solver._init_convenience_accessors()
solver._init_quad_fields()
solver._get_active_terms()
solver._build_jit_functions()
solver._build_terms()

# Now we can access assembly_layout with ORIGINAL global indices (before PETSc)
layout = solver.assembly_layout
coo = layout.matrix_coo

# Store copies of original global indices before PETSc reorders them
original_global_rows = coo.global_rows.copy()
original_global_cols = coo.global_cols.copy()

# Now complete initialization (this will modify global indices in place)
solver._init_petsc()
solver.update_quad()
solver.update_output_fields()

print(f"Inner grid: {solver.grid_idx.Nx_inner} x {solver.grid_idx.Ny_inner}")
print(f"Inner points: {solver.nb_inner_pts}")
print(f"Variables: {solver.variables}")
print(f"Residuals: {solver.residuals}")

## Grid Index Masks

The `GridIndexManager` creates index masks that map grid coordinates to solution vector indices.

For a 2×2 inner grid with ghost cells, we have a 4×4 padded grid.

The mask is displayed with y increasing upward (like a plot), so:
- Bottom row = y=0 (South ghost)
- Top row = y=3 (North ghost)
- Left column = x=0 (West ghost)
- Right column = x=3 (East ghost)

In [ ]:
grid_idx = solver.grid_idx

print("Index mask (inner points only):")
print("Rows = x-index, Columns = y-index")
print("-1 = ghost/boundary cell")
print()
# Transpose for display: rows=y (bottom to top), cols=x
mask_inner = grid_idx.index_mask_inner_local
print(np.flipud(mask_inner.T))

In [ ]:
print("Index mask (with ghost points for contributors):")
mask_padded = grid_idx.index_mask_padded_local('')
print(np.flipud(mask_padded.T))
print()
print(f"Total contributors: {grid_idx.nb_contributors}")

## Square Elements and Triangles

Each square cell is split into 2 triangles:
- **Left triangle**: corners [bl, tl, br]
- **Right triangle**: corners [tr, br, tl]

With a 2×2 inner grid, we have 3×3 = 9 squares.

In [ ]:
print(f"Number of squares: {grid_idx.nb_sq}")
print(f"Squares per row (x): {grid_idx.sq_per_row}")
print(f"Squares per col (y): {grid_idx.sq_per_col}")
print()

# Show square corners (TO = inner point indices)
print("Square corners (inner point indices):")
print("Columns: [bl, br, tl, tr]")
print("-1 means corner is outside inner domain")
print()
sq_TO = grid_idx.sq_TO_inner
for sq_idx in range(grid_idx.nb_sq):
    x, y = grid_idx.sq_x_arr[sq_idx], grid_idx.sq_y_arr[sq_idx]
    corners = sq_TO[sq_idx]
    print(f"Square ({x},{y}): bl={corners[0]:2}, br={corners[1]:2}, tl={corners[2]:2}, tr={corners[3]:2}")

## COO Pattern

The sparse matrix uses COO (Coordinate) format: parallel arrays of (row, col, value).

The pattern is determined by the 7-point stencil (each inner point connects to itself and 6 neighbors via triangular elements).

In [ ]:
layout = solver.assembly_layout
coo = layout.matrix_coo

print(f"Total non-zeros: {coo.nnz}")
print(f"Matrix size: {layout.nb_vars * solver.nb_inner_pts} x {layout.nb_vars * grid_idx.nb_contributors}")
print()

# Build point index to (x,y) coordinate mapping
mask_inner = grid_idx.index_mask_inner_local
mask_padded = grid_idx.index_mask_padded_local('')

def pt_to_xy_inner(pt_idx):
    """Convert inner point index to (x,y) grid coordinates."""
    coords = np.argwhere(mask_inner == pt_idx)
    if len(coords) > 0:
        return (int(coords[0][0]), int(coords[0][1]))
    return None

def pt_to_xy_padded(pt_idx):
    """Convert contributor point index to (x,y) grid coordinates."""
    coords = np.argwhere(mask_padded == pt_idx)
    if len(coords) > 0:
        return (int(coords[0][0]), int(coords[0][1]))
    return None

# Helper to decode row/col indices
def decode_row(row_idx, nb_inner, residuals):
    """Decode row index to (residual_name, inner_point)."""
    res_idx = row_idx // nb_inner
    pt_idx = row_idx % nb_inner
    return residuals[res_idx], pt_idx

def decode_col(col_idx, nb_contrib, variables):
    """Decode column index to (variable_name, contributor_point)."""
    var_idx = col_idx // nb_contrib
    pt_idx = col_idx % nb_contrib
    return variables[var_idx], pt_idx

# Show first entries with decoded meaning
print("First 20 COO entries with decoded meaning:")
print("idx | row | col | residual     | pt (x,y) | variable | pt (x,y)")
print("-" * 68)
for i in range(min(20, coo.nnz)):
    row = coo.local_rows[i]
    col = coo.local_cols[i]
    res_name, res_pt = decode_row(row, solver.nb_inner_pts, solver.residuals)
    var_name, var_pt = decode_col(col, grid_idx.nb_contributors, solver.variables)
    res_xy = pt_to_xy_inner(res_pt)
    var_xy = pt_to_xy_padded(var_pt)
    print(f"{i:3} | {row:3} | {col:3} | {res_name:12} | {str(res_xy):8} | {var_name:8} | {str(var_xy):8}")

### Local vs Global Index Ordering

The local and global indices use different ordering schemes:

- **Local**: Block ordering by variable/residual (all rho entries, then all jx, then all jy)
- **Global**: Point-interleaved ordering (all variables for point 0, then all for point 1, ...)

This is important for MPI parallelization where each rank owns contiguous global indices.

**Note:** We captured the original global indices before PETSc initialization. PETSc's `setPreallocationCOO` reorders these arrays in place for efficient assembly, but we stored copies of the original point-interleaved layout.

In [ ]:
# Local vs Global index ordering formulas
nb_inner = solver.nb_inner_pts
nb_contrib = grid_idx.nb_contributors
nb_vars = layout.nb_vars
nb_res = len(solver.residuals)

print("Index ordering comparison:")
print("=" * 80)
print()
print("LOCAL ordering (block by variable/residual):")
print(f"  Rows: [mass_pt0, mass_pt1, ..., mass_pt{nb_inner-1}, mom_x_pt0, ..., mom_y_pt{nb_inner-1}]")
print(f"  Cols: [rho_pt0, rho_pt1, ..., rho_pt{nb_contrib-1}, jx_pt0, ..., jy_pt{nb_contrib-1}]")
print()
print("GLOBAL ordering (point-interleaved):")
print(f"  Rows: [mass_pt0, mom_x_pt0, mom_y_pt0, mass_pt1, mom_x_pt1, mom_y_pt1, ...]")
print(f"  Cols: [rho_pt0, jx_pt0, jy_pt0, rho_pt1, jx_pt1, jy_pt1, ...]")
print()

# Show conversion formulas
print("Conversion formulas:")
print(f"  local_row  = res_idx * nb_inner + pt_idx")
print(f"  global_row = global_pt * nb_res + res_idx")
print()
print(f"  local_col  = var_idx * nb_contrib + pt_idx")
print(f"  global_col = global_pt * nb_vars + var_idx")
print()
print("where global_pt = l2g_pts[local_pt] (local-to-global point mapping)")
print("In serial mode, global_pt == local_pt.")

In [ ]:
# Show local indices alongside original global indices (before PETSc reordering)
print("Local vs Global index mapping (first 20 COO entries):")
print("=" * 95)
print()
print("idx | local_row | local_col | global_row | global_col | meaning")
print("-" * 95)

for i in range(min(20, coo.nnz)):
    l_row = coo.local_rows[i]
    l_col = coo.local_cols[i]
    g_row = original_global_rows[i]
    g_col = original_global_cols[i]
    
    # Decode local indices
    res_name, res_pt = decode_row(l_row, solver.nb_inner_pts, solver.residuals)
    var_name, var_pt = decode_col(l_col, grid_idx.nb_contributors, solver.variables)
    
    meaning = f"{res_name}@pt{res_pt} <- {var_name}@pt{var_pt}"
    is_diag = " (diag)" if g_row == g_col else ""
    print(f"{i:3} | {l_row:9} | {l_col:9} | {g_row:10} | {g_col:10} | {meaning}{is_diag}")

print()
print("Self-connections appear on the global diagonal (global_row == global_col):")
print("  e.g., mass@pt2 <- rho@pt2 has global indices (6, 6)")

## Stencil Connectivity

The 7-point stencil shows which inner points connect to which contributor points.

In [ ]:
inner_pts, contrib_pts = grid_idx.get_stencil_connectivity()

print("Stencil connectivity:")
print("inner_pt → contrib_pt")
print("-" * 25)
for i, c in zip(inner_pts, contrib_pts):
    print(f"    {i}    →     {c}")

## Term Maps

Each physics term has a precomputed map telling the assembler:
1. Which COO entries to update (`coo_idx`)
2. What integration weights to use (`weights`)
3. Which quadrature point provides the field values (`quad_idx`)
4. Which square element to look up (`sq_x`, `sq_y`)

### Term Naming Convention

Term names follow the pattern `R{eq}{term}{dir}`:
| Term | Equation | Description |
|------|----------|-------------|
| `R11x` | mass | x-flux divergence (∂jx/∂x) |
| `R11y` | mass | y-flux divergence (∂jy/∂y) |
| `R21x` | momentum_x | x-pressure gradient (∂p/∂x) |
| `R21y` | momentum_y | y-pressure gradient (∂p/∂y) |
| `R24x` | momentum_x | x-wall shear stress |
| `R24y` | momentum_y | y-wall shear stress |

### Quadrature Point Indices

Each square is split into 2 triangles with 3 quadrature points each:
- `quad_idx 0, 1, 2` → Left triangle (corners: bl, tl, br)
- `quad_idx 3, 4, 5` → Right triangle (corners: tr, br, tl)

In [ ]:
print("Available Jacobian term keys:")
for key in sorted(layout.jacobian_terms.keys()):
    term_map = layout.jacobian_terms[key]
    print(f"  {key}: {len(term_map.coo_idx)} contributions")

In [ ]:
print("Available RHS term keys:")
for key in sorted(layout.rhs_terms.keys()):
    term_map = layout.rhs_terms[key]
    print(f"  {key}: {len(term_map.output_idx)} contributions")

## Examining a Single Term Map

Let's look at one term in detail to understand how assembly works.

In [ ]:
# Pick a Jacobian term: R11x contributes to mass equation, depends on jx
key = ('R11x', 'mass', 'jx')
if key in layout.jacobian_terms:
    jac_map = layout.jacobian_terms[key]
    
    print(f"Jacobian term: {key}")
    print(f"Number of contributions: {len(jac_map.coo_idx)}")
    print()
    print("Detailed contributions:")
    print("  i | coo_idx | weight | quad | sq_x | sq_y | sign")
    print("-" * 55)
    
    n_show = min(15, len(jac_map.coo_idx))
    for i in range(n_show):
        sign = jac_map.signs[i] if jac_map.signs is not None else "N/A"
        print(f"{i:3} | {jac_map.coo_idx[i]:7} | {jac_map.weights[i]:6.4f} | {jac_map.quad_idx[i]:4} | {jac_map.sq_x[i]:4} | {jac_map.sq_y[i]:4} | {sign}")
    if len(jac_map.coo_idx) > n_show:
        print(f"... ({len(jac_map.coo_idx) - n_show} more entries)")
else:
    print(f"Term {key} not found")

In [ ]:
# Pick an RHS term
key = ('R11x', 'mass')
if key in layout.rhs_terms:
    rhs_map = layout.rhs_terms[key]
    
    print(f"RHS term: {key}")
    print(f"Number of contributions: {len(rhs_map.output_idx)}")
    print()
    print("Detailed contributions:")
    print("  i | out_idx | weight | quad | sq_x | sq_y")
    print("-" * 48)
    
    n_show = min(15, len(rhs_map.output_idx))
    for i in range(n_show):
        print(f"{i:3} | {rhs_map.output_idx[i]:7} | {rhs_map.weights[i]:6.4f} | {rhs_map.quad_idx[i]:4} | {rhs_map.sq_x[i]:4} | {rhs_map.sq_y[i]:4}")
    if len(rhs_map.output_idx) > n_show:
        print(f"... ({len(rhs_map.output_idx) - n_show} more entries)")
else:
    print(f"Term {key} not found")

## Assembly Process

The actual assembly is simple once the layout is built:

```python
# Jacobian assembly (simplified)
for term in terms:
    for dep_var in term.dep_vars:
        idx = layout.jacobian_terms[(term.name, term.res, dep_var)]
        
        # Get field values at quadrature points
        field_vals = quad_fields[idx.sq_x, idx.sq_y, idx.quad_idx]
        
        # Compute contribution
        contrib = idx.weights * field_vals
        if idx.signs is not None:
            contrib *= idx.signs
        
        # Add to COO values
        coo_values[idx.coo_idx] += contrib
```

This is O(nnz) - we only touch non-zero entries!

## Visualizing the Sparsity Pattern

In [ ]:
import matplotlib.pyplot as plt

# Create dense matrix from COO for visualization
n_rows = layout.nb_vars * solver.nb_inner_pts
n_cols = layout.nb_vars * grid_idx.nb_contributors

dense = np.zeros((n_rows, n_cols))
for i in range(coo.nnz):
    dense[coo.local_rows[i], coo.local_cols[i]] = 1

fig, ax = plt.subplots(figsize=(8, 6))
ax.spy(dense, markersize=3)
ax.set_xlabel('Column (variables × contributors)')
ax.set_ylabel('Row (residuals × inner points)')
ax.set_title(f'Jacobian sparsity pattern ({coo.nnz} non-zeros)')

# Add block separators
nb_inner = solver.nb_inner_pts
nb_contrib = grid_idx.nb_contributors
for i in range(1, layout.nb_vars):
    ax.axhline(i * nb_inner - 0.5, color='red', linewidth=0.5, linestyle='--')
    ax.axvline(i * nb_contrib - 0.5, color='red', linewidth=0.5, linestyle='--')

plt.tight_layout()
plt.show()

## Summary

The assembly layout architecture enables efficient sparse matrix assembly:

1. **Precomputation**: All index mappings are computed once during setup
2. **O(nnz) assembly**: Only non-zero entries are touched during each Newton iteration
3. **Vectorized**: NumPy array operations instead of nested loops
4. **MPI-ready**: Global indices enable distributed assembly with PETSc

### Key Index Relationships

| Index Type | Description | Range |
|------------|-------------|-------|
| Inner point | Points where we solve | 0 to nb_inner_pts-1 |
| Contributor | Points that contribute to assembly | 0 to nb_contributors-1 |
| COO index | Position in sparse values array | 0 to nnz-1 |
| Square index | Which element cell | 0 to nb_sq-1 |
| Quad index | Which quadrature point | 0-2 (left tri) or 3-5 (right tri) |